In [2]:
import pygame
import pygame_gui
from pygame_gui.elements import UIButton
import random

pygame-ce 2.3.0 (SDL 2.26.4, Python 3.9.12)


In [1]:
!pip install pygame_gui --user

  Using cached pygame_gui-0.6.9-py3-none-any.whl (23.6 MB)
  Using cached pygame_ce-2.3.0-cp39-cp39-win_amd64.whl (11.5 MB)


In [3]:
# Game constants
SCREEN_WIDTH = 480
SCREEN_HEIGHT = 350
GRID_WIDTH = 6
GRID_HEIGHT = 3
PADDLE_WIDTH = 130
PADDLE_HEIGHT = 15
BRICK_WIDTH = 80
BRICK_HEIGHT = 20
BALL_RADIUS = 7
BALL_SPEED_X = random.choice([-2, -1, 1, 2])
BALL_SPEED_Y = 2
MAX_PADDLE_SPEED = 3
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)

In [4]:
# Create the bricks
def create_bricks():
    bricks = []
    brick_colors = [(255, 255, 255), (0, 150, 110), (214, 38, 18)]
    for row in range(GRID_HEIGHT):
        row_color = brick_colors[row % len(brick_colors)]
        for col in range(GRID_WIDTH):
            brick = pygame.Rect(col * BRICK_WIDTH, row * BRICK_HEIGHT + 30, BRICK_WIDTH, BRICK_HEIGHT)
            bricks.append((brick, row_color))
    return bricks

# Initialize Pygame
pygame.init()
pygame.display.set_caption("Breakout")
window_surface = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
manager = pygame_gui.UIManager((SCREEN_WIDTH, SCREEN_HEIGHT))

# Create the paddle
paddle = pygame.Rect((SCREEN_WIDTH - PADDLE_WIDTH) // 2, SCREEN_HEIGHT - PADDLE_HEIGHT - 10, PADDLE_WIDTH, PADDLE_HEIGHT)
paddle_speed = 0

# Create the ball
ball = pygame.Rect(SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2, BALL_RADIUS, BALL_RADIUS)

# Create the bricks
bricks = create_bricks()

# Game variables
is_running = True
is_game_started = False
is_ball_in_motion = False
is_ball_fallen = False
score = 0
max_score = 0
start_time = 0
final_score = 0
display_final_score = False
clock = pygame.time.Clock()

# Create a font object for rendering text
font = pygame.font.Font(None, 24)

# Game loop
while is_running:
    time_delta = clock.tick(60) / 1000.0

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            is_running = False

        manager.process_events(event)

        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_LEFT:
                paddle_speed = max(paddle_speed - 2, -MAX_PADDLE_SPEED)
            elif event.key == pygame.K_RIGHT:
                paddle_speed = min(paddle_speed + 2, MAX_PADDLE_SPEED)
            elif event.key == pygame.K_SPACE:
                if not is_game_started or (is_ball_fallen and not is_ball_in_motion):
                    if len(bricks) == 0:
                        score = 0  # Reset the score to zero
                    is_game_started = True
                    is_ball_in_motion = True
                    is_ball_fallen = False
                    ball.center = (SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2)
                    bricks = create_bricks()
                    start_time = pygame.time.get_ticks()  # Start the timer

        if event.type == pygame.KEYUP:
            if event.key == pygame.K_LEFT or event.key == pygame.K_RIGHT:
                paddle_speed = 0

    if is_game_started:
        paddle.x += paddle_speed

        if paddle.left < 0:
            paddle.left = 0
        elif paddle.right > SCREEN_WIDTH:
            paddle.right = SCREEN_WIDTH

        if is_ball_in_motion:
            ball.x += BALL_SPEED_X
            ball.y += BALL_SPEED_Y

            if ball.left <= 0 or ball.right >= SCREEN_WIDTH:
                BALL_SPEED_X = -BALL_SPEED_X
            if ball.top <= 0:
                BALL_SPEED_Y = -BALL_SPEED_Y

            if ball.colliderect(paddle):
                ball_dir = ball.x - paddle.x
                if ball_dir < PADDLE_WIDTH // 5:
                    BALL_SPEED_X = -2
                elif ball_dir < 2 * PADDLE_WIDTH // 5:
                    BALL_SPEED_X = -1
                elif ball_dir < 3 * PADDLE_WIDTH // 5:
                    BALL_SPEED_X = 0
                elif ball_dir < 4 * PADDLE_WIDTH // 5:
                    BALL_SPEED_X = 1
                else:
                    BALL_SPEED_X = 2
                BALL_SPEED_Y = -BALL_SPEED_Y

            for brick, color in bricks:
                if ball.colliderect(brick):
                    bricks.remove((brick, color))
                    BALL_SPEED_Y = -BALL_SPEED_Y
                    score += 10

            if ball.top >= SCREEN_HEIGHT:
                is_ball_fallen = True
                score = 0  # Reset the score to zero

            if len(bricks) == 0 or is_ball_fallen:
                is_game_started = False
                is_ball_in_motion = False
                paddle.centerx = SCREEN_WIDTH // 2
                paddle_speed = 0
                BALL_SPEED_X = random.choice([-2, -1, 1, 2])
                BALL_SPEED_Y = 2
                if len(bricks) == 0:
                    final_score = score + (score - (pygame.time.get_ticks() - start_time) // 1000)
                else:
                    final_score = 0
                if final_score > max_score:
                    max_score = final_score
                display_final_score = True
                final_score_start_time = pygame.time.get_ticks()

    window_surface.fill(BLACK)

    pygame.draw.rect(window_surface, WHITE, paddle)
    pygame.draw.circle(window_surface, WHITE, (ball.x, ball.y), BALL_RADIUS)

    for brick, color in bricks:
        pygame.draw.rect(window_surface, color, brick)
        pygame.draw.rect(window_surface, BLACK, brick, 1)

    score_text = f"Score: {score}"
    max_score_text = f"Max Score: {max_score}"
    score_surface = font.render(score_text, True, WHITE)
    max_score_surface = font.render(max_score_text, True, WHITE)
    score_position = (10, 10)
    max_score_position = (score_position[0] + score_surface.get_width() + 10, score_position[1])
    window_surface.blit(score_surface, score_position)
    window_surface.blit(max_score_surface, max_score_position)

    if is_game_started:
        elapsed_time = (pygame.time.get_ticks() - start_time) // 1000
        timer_text = f"Time: {elapsed_time} sec"
        timer_surface = font.render(timer_text, True, WHITE)
        window_surface.blit(timer_surface, (SCREEN_WIDTH - timer_surface.get_width() - 10, 10))

    if display_final_score:
        final_score_text = f"Final Score: {final_score}"
        final_score_surface = font.render(final_score_text, True, WHITE)
        window_surface.blit(final_score_surface, (SCREEN_WIDTH // 2 - final_score_surface.get_width() // 2, SCREEN_HEIGHT // 2))
        if pygame.time.get_ticks() - final_score_start_time > 3000:
            display_final_score = False

    manager.update(time_delta)
    manager.draw_ui(window_surface)

    pygame.display.update()

pygame.display.quit()
pygame.quit()

In [5]:
# Game constants
SCREEN_WIDTH = 480
SCREEN_HEIGHT = 400
GRID_WIDTH = 4
GRID_HEIGHT = 4
PADDLE_WIDTH = 200
PADDLE_HEIGHT = 15
BRICK_WIDTH = 120
BRICK_HEIGHT = 20
BALL_RADIUS = 7
BALL_SPEED_X = random.choice([-2, -1, 1, 2])
BALL_SPEED_Y = 2
MAX_PADDLE_SPEED = 3
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)

In [6]:
# Create the bricks
def create_bricks():
    bricks = []
    brick_colors = [(217, 0, 18), (0, 20, 137), (255, 158, 27), (0, 43, 127)]
    for row in range(GRID_HEIGHT):
        row_color = brick_colors[row % len(brick_colors)]
        for col in range(GRID_WIDTH):
            brick = pygame.Rect(col * BRICK_WIDTH, row * BRICK_HEIGHT + 30, BRICK_WIDTH, BRICK_HEIGHT)
            bricks.append((brick, row_color))
    return bricks

# Initialize Pygame
pygame.init()
pygame.display.set_caption("Breakout")
window_surface = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
manager = pygame_gui.UIManager((SCREEN_WIDTH, SCREEN_HEIGHT))

# Create the paddle
paddle = pygame.Rect((SCREEN_WIDTH - PADDLE_WIDTH) // 2, SCREEN_HEIGHT - PADDLE_HEIGHT - 10, PADDLE_WIDTH, PADDLE_HEIGHT)
paddle_speed = 0

# Create the ball
ball = pygame.Rect(SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2, BALL_RADIUS, BALL_RADIUS)

# Create the bricks
bricks = create_bricks()

# Game variables
is_running = True
is_game_started = False
is_ball_in_motion = False
is_ball_fallen = False
score = 0
max_score = 0
start_time = 0
final_score = 0
display_final_score = False
clock = pygame.time.Clock()

# Create a font object for rendering text
font = pygame.font.Font(None, 24)

# Game loop
while is_running:
    time_delta = clock.tick(60) / 1000.0

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            is_running = False

        manager.process_events(event)

        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_LEFT:
                paddle_speed = max(paddle_speed - 2, -MAX_PADDLE_SPEED)
            elif event.key == pygame.K_RIGHT:
                paddle_speed = min(paddle_speed + 2, MAX_PADDLE_SPEED)
            elif event.key == pygame.K_SPACE:
                if not is_game_started or (is_ball_fallen and not is_ball_in_motion):
                    if len(bricks) == 0:
                        score = 0  # Reset the score to zero
                    is_game_started = True
                    is_ball_in_motion = True
                    is_ball_fallen = False
                    ball.center = (SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2)
                    bricks = create_bricks()
                    start_time = pygame.time.get_ticks()  # Start the timer

        if event.type == pygame.KEYUP:
            if event.key == pygame.K_LEFT or event.key == pygame.K_RIGHT:
                paddle_speed = 0

    if is_game_started:
        paddle.x += paddle_speed

        if paddle.left < 0:
            paddle.left = 0
        elif paddle.right > SCREEN_WIDTH:
            paddle.right = SCREEN_WIDTH

        if is_ball_in_motion:
            ball.x += BALL_SPEED_X
            ball.y += BALL_SPEED_Y

            if ball.left <= 0 or ball.right >= SCREEN_WIDTH:
                BALL_SPEED_X = -BALL_SPEED_X
            if ball.top <= 0:
                BALL_SPEED_Y = -BALL_SPEED_Y

            if ball.colliderect(paddle):
                ball_dir = ball.x - paddle.x
                if ball_dir < PADDLE_WIDTH // 5:
                    BALL_SPEED_X = -2
                elif ball_dir < 2 * PADDLE_WIDTH // 5:
                    BALL_SPEED_X = -1
                elif ball_dir < 3 * PADDLE_WIDTH // 5:
                    BALL_SPEED_X = 0
                elif ball_dir < 4 * PADDLE_WIDTH // 5:
                    BALL_SPEED_X = 1
                else:
                    BALL_SPEED_X = 2
                BALL_SPEED_Y = -BALL_SPEED_Y

            for brick, color in bricks:
                if ball.colliderect(brick):
                    bricks.remove((brick, color))
                    BALL_SPEED_Y = -BALL_SPEED_Y
                    score += 10

            if ball.top >= SCREEN_HEIGHT:
                is_ball_fallen = True
                score = 0  # Reset the score to zero

            if len(bricks) == 0 or is_ball_fallen:
                is_game_started = False
                is_ball_in_motion = False
                paddle.centerx = SCREEN_WIDTH // 2
                paddle_speed = 0
                BALL_SPEED_X = random.choice([-2, -1, 1, 2])
                BALL_SPEED_Y = 2
                if len(bricks) == 0:
                    final_score = score + (score - (pygame.time.get_ticks() - start_time) // 1000)
                else:
                    final_score = 0
                if final_score > max_score:
                    max_score = final_score
                display_final_score = True
                final_score_start_time = pygame.time.get_ticks()

    window_surface.fill(BLACK)

    pygame.draw.rect(window_surface, WHITE, paddle)
    pygame.draw.circle(window_surface, WHITE, (ball.x, ball.y), BALL_RADIUS)

    for brick, color in bricks:
        pygame.draw.rect(window_surface, color, brick)
        pygame.draw.rect(window_surface, BLACK, brick, 1)

    score_text = f"Score: {score}"
    max_score_text = f"Max Score: {max_score}"
    score_surface = font.render(score_text, True, WHITE)
    max_score_surface = font.render(max_score_text, True, WHITE)
    score_position = (10, 10)
    max_score_position = (score_position[0] + score_surface.get_width() + 10, score_position[1])
    window_surface.blit(score_surface, score_position)
    window_surface.blit(max_score_surface, max_score_position)

    if is_game_started:
        elapsed_time = (pygame.time.get_ticks() - start_time) // 1000
        timer_text = f"Time: {elapsed_time} sec"
        timer_surface = font.render(timer_text, True, WHITE)
        window_surface.blit(timer_surface, (SCREEN_WIDTH - timer_surface.get_width() - 10, 10))

    if display_final_score:
        final_score_text = f"Final Score: {final_score}"
        final_score_surface = font.render(final_score_text, True, WHITE)
        window_surface.blit(final_score_surface, (SCREEN_WIDTH // 2 - final_score_surface.get_width() // 2, SCREEN_HEIGHT // 2))
        if pygame.time.get_ticks() - final_score_start_time > 3000:
            display_final_score = False

    manager.update(time_delta)
    manager.draw_ui(window_surface)

    pygame.display.update()

pygame.display.quit()
pygame.quit()

In [7]:
# Game constants
SCREEN_WIDTH = 480
SCREEN_HEIGHT = 350
GRID_WIDTH = 6
GRID_HEIGHT = 6
PADDLE_WIDTH = 130
PADDLE_HEIGHT = 15
BRICK_WIDTH = 80
BRICK_HEIGHT = 20
BALL_RADIUS = 7
BALL_SPEED_X = random.choice([-2, -1, 1, 2])
BALL_SPEED_Y = 2
MAX_PADDLE_SPEED = 3
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)

In [8]:
# Create the bricks
def create_bricks():
    bricks = []
    brick_colors = [(255, 130, 0), (255, 255, 255), (0, 154, 68), (0, 43, 127), (252, 209, 22), (206, 17, 38)]
    for row in range(GRID_HEIGHT):
        row_color = brick_colors[row % len(brick_colors)]
        for col in range(GRID_WIDTH - row):
            brick_x = col * BRICK_WIDTH + (SCREEN_WIDTH - (BRICK_WIDTH * (GRID_WIDTH - row))) // 2
            brick_y = row * BRICK_HEIGHT + 30
            brick = pygame.Rect(brick_x, brick_y, BRICK_WIDTH, BRICK_HEIGHT)
            bricks.append((brick, row_color))
    return bricks

# Initialize Pygame
pygame.init()
pygame.display.set_caption("Breakout")
window_surface = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
manager = pygame_gui.UIManager((SCREEN_WIDTH, SCREEN_HEIGHT))

# Create the paddle
paddle = pygame.Rect((SCREEN_WIDTH - PADDLE_WIDTH) // 2, SCREEN_HEIGHT - PADDLE_HEIGHT - 10, PADDLE_WIDTH, PADDLE_HEIGHT)
paddle_speed = 0

# Create the ball
ball = pygame.Rect(SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2, BALL_RADIUS, BALL_RADIUS)

# Create the bricks
bricks = create_bricks()

# Game variables
is_running = True
is_game_started = False
is_ball_in_motion = False
is_ball_fallen = False
score = 0
max_score = 0
start_time = 0
final_score = 0
display_final_score = False
clock = pygame.time.Clock()

# Create a font object for rendering text
font = pygame.font.Font(None, 24)

# Game loop
while is_running:
    time_delta = clock.tick(60) / 1000.0

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            is_running = False

        manager.process_events(event)

        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_LEFT:
                paddle_speed = max(paddle_speed - 2, -MAX_PADDLE_SPEED)
            elif event.key == pygame.K_RIGHT:
                paddle_speed = min(paddle_speed + 2, MAX_PADDLE_SPEED)
            elif event.key == pygame.K_SPACE:
                if not is_game_started or (is_ball_fallen and not is_ball_in_motion):
                    if len(bricks) == 0:
                        score = 0  # Reset the score to zero
                    is_game_started = True
                    is_ball_in_motion = True
                    is_ball_fallen = False
                    ball.center = (SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2)
                    bricks = create_bricks()
                    start_time = pygame.time.get_ticks()  # Start the timer

        if event.type == pygame.KEYUP:
            if event.key == pygame.K_LEFT or event.key == pygame.K_RIGHT:
                paddle_speed = 0

    if is_game_started:
        paddle.x += paddle_speed

        if paddle.left < 0:
            paddle.left = 0
        elif paddle.right > SCREEN_WIDTH:
            paddle.right = SCREEN_WIDTH

        if is_ball_in_motion:
            ball.x += BALL_SPEED_X
            ball.y += BALL_SPEED_Y

            if ball.left <= 0 or ball.right >= SCREEN_WIDTH:
                BALL_SPEED_X = -BALL_SPEED_X
            if ball.top <= 0:
                BALL_SPEED_Y = -BALL_SPEED_Y

            if ball.colliderect(paddle):
                ball_dir = ball.x - paddle.x
                if ball_dir < PADDLE_WIDTH // 5:
                    BALL_SPEED_X = -2
                elif ball_dir < 2 * PADDLE_WIDTH // 5:
                    BALL_SPEED_X = -1
                elif ball_dir < 3 * PADDLE_WIDTH // 5:
                    BALL_SPEED_X = 0
                elif ball_dir < 4 * PADDLE_WIDTH // 5:
                    BALL_SPEED_X = 1
                else:
                    BALL_SPEED_X = 2
                BALL_SPEED_Y = -BALL_SPEED_Y

            for brick, color in bricks:
                if ball.colliderect(brick):
                    bricks.remove((brick, color))
                    BALL_SPEED_Y = -BALL_SPEED_Y
                    score += 10

            if ball.top >= SCREEN_HEIGHT:
                is_ball_fallen = True
                score = 0  # Reset the score to zero

            if len(bricks) == 0 or is_ball_fallen:
                is_game_started = False
                is_ball_in_motion = False
                paddle.centerx = SCREEN_WIDTH // 2
                paddle_speed = 0
                BALL_SPEED_X = random.choice([-2, -1, 1, 2])
                BALL_SPEED_Y = 2
                if len(bricks) == 0:
                    final_score = score + (score - (pygame.time.get_ticks() - start_time) // 1000)
                else:
                    final_score = 0
                if final_score > max_score:
                    max_score = final_score
                display_final_score = True
                final_score_start_time = pygame.time.get_ticks()

    window_surface.fill(BLACK)

    pygame.draw.rect(window_surface, WHITE, paddle)
    pygame.draw.circle(window_surface, WHITE, (ball.x, ball.y), BALL_RADIUS)

    for brick, color in bricks:
        pygame.draw.rect(window_surface, color, brick)
        pygame.draw.rect(window_surface, BLACK, brick, 1)

    score_text = f"Score: {score}"
    max_score_text = f"Max Score: {max_score}"
    score_surface = font.render(score_text, True, WHITE)
    max_score_surface = font.render(max_score_text, True, WHITE)
    score_position = (10, 10)
    max_score_position = (score_position[0] + score_surface.get_width() + 10, score_position[1])
    window_surface.blit(score_surface, score_position)
    window_surface.blit(max_score_surface, max_score_position)

    if is_game_started:
        elapsed_time = (pygame.time.get_ticks() - start_time) // 1000
        timer_text = f"Time: {elapsed_time} sec"
        timer_surface = font.render(timer_text, True, WHITE)
        window_surface.blit(timer_surface, (SCREEN_WIDTH - timer_surface.get_width() - 10, 10))

    if display_final_score:
        final_score_text = f"Final Score: {final_score}"
        final_score_surface = font.render(final_score_text, True, WHITE)
        window_surface.blit(final_score_surface, (SCREEN_WIDTH // 2 - final_score_surface.get_width() // 2, SCREEN_HEIGHT // 2))
        if pygame.time.get_ticks() - final_score_start_time > 3000:
            display_final_score = False

    manager.update(time_delta)
    manager.draw_ui(window_surface)

    pygame.display.update()

pygame.display.quit()
pygame.quit()